In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import spacy
import re
import nltk
from spacy.tokenizer import Tokenizer
import pickle
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
from nltk import word_tokenize

In [3]:
import csv
import re
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [4]:
dt = pd.read_csv('train.csv')

In [5]:
dt.dropna()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


# English

In [6]:
# english data set
df_en = dt.loc[dt['lang_abv'] == 'en']
df_en

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
7,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2
8,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [7]:
#create a csv file
df_en.to_csv('df_en.csv', encoding='utf-8', index = False)

In [8]:
#load the csv file to create the input data for Doc2vec
data = list()
words = list()
with open('df_en.csv', encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile)
    for row in reader:
        sent = [word_tokenize(row[i].lower()) for i in range(1,3)]
        words = sent[0]+ sent[1]
        #print(words)
        data.append(TaggedDocument(words,[row[0]]))

In [28]:
data

[TaggedDocument(words=['premise', 'hypothesis'], tags=['id']),
 TaggedDocument(words=['and', 'these', 'comments', 'were', 'considered', 'in', 'formulating', 'the', 'interim', 'rules', '.', 'the', 'rules', 'developed', 'in', 'the', 'interim', 'were', 'put', 'together', 'with', 'these', 'comments', 'in', 'mind', '.'], tags=['5130fd2cb5']),
 TaggedDocument(words=['these', 'are', 'issues', 'that', 'we', 'wrestle', 'with', 'in', 'practice', 'groups', 'of', 'law', 'firms', ',', 'she', 'said', '.', 'practice', 'groups', 'are', 'not', 'permitted', 'to', 'work', 'on', 'these', 'issues', '.'], tags=['5b72532a0b']),
 TaggedDocument(words=['you', 'know', 'they', 'ca', "n't", 'really', 'defend', 'themselves', 'like', 'somebody', 'grown', 'uh', 'say', 'my', 'age', 'you', 'know', 'yeah', 'they', 'ca', "n't", 'defend', 'themselves', 'because', 'of', 'their', 'age', '.'], tags=['5622f0c60b']),
 TaggedDocument(words=['from', 'cockpit', 'country', 'to', 'st.', 'ann', "'s", 'bay', 'from', 'st.', 'ann', "'

In [9]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [10]:
from gensim.models import Doc2Vec
model = Doc2Vec(data,dm =1,vector_size=20, min_count=1, alpha = 0.1, min_alpha = 0.02, negative=0)

2022-08-24 16:14:51,099 : INFO : collecting all words and their counts
2022-08-24 16:14:51,102 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-08-24 16:14:51,221 : INFO : collected 12970 word types and 6871 unique tags from a corpus of 6871 examples and 226803 words
2022-08-24 16:14:51,222 : INFO : Creating a fresh vocabulary
2022-08-24 16:14:51,276 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 12970 unique words (100.0%% of original 12970, drops 0)', 'datetime': '2022-08-24T16:14:51.276990', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'prepare_vocab'}
2022-08-24 16:14:51,277 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 226803 word corpus (100.0%% of original 226803, drops 0)', 'datetime': '2022-08-24T16:14:51.277990', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1

In [34]:
model.dv['5130fd2cb5']

array([-0.252591  ,  0.24796964,  0.43360314,  0.02377016, -0.11134874,
        0.3133653 ,  0.12632562,  0.20351757, -0.00663013, -0.14751585,
       -0.06196272, -0.47113913,  0.16508657,  0.12605524, -0.168435  ,
       -0.74205077,  0.16500331, -0.31059596,  0.13752033, -0.3448493 ,
        0.09398891,  0.03876414, -0.15376937,  0.04769925, -0.28010607,
       -0.10922952, -0.290214  , -0.20648438, -0.11248926, -0.09509275,
        0.15195404,  0.39841092, -0.23087122,  0.00277491, -0.35637093,
        0.32186258, -0.13623077, -0.10241799, -0.05899918, -0.55689347,
        0.523616  , -0.3677814 ,  0.07881328,  0.24454647, -0.05729768,
        0.04946067,  0.07007241, -0.07159048, -0.06843924,  0.10901473,
        0.39866894,  0.10043395,  0.30700472,  0.02715515,  0.01228601,
       -0.00224675,  0.02938785, -0.06174698, -0.22971325, -0.1463519 ,
        0.14684963,  0.00122315,  0.09731619,  0.05339675, -0.13652897,
       -0.03201896, -0.14983171,  0.01077024, -0.3556333 , -0.06

In [19]:
model.dv

In [11]:
x_train = [model.dv[i] for i in df_en['id'] ]

In [32]:
x_train

[array([-0.09476398,  0.41510674, -0.13955067,  0.21966042,  0.05958949,
        -0.40455118,  0.30478093,  0.15266271, -0.57785374, -0.3138074 ,
        -0.3063862 , -0.5123052 , -0.06117177, -0.20716242,  0.40503323,
        -0.15069483,  0.3529847 ,  0.17792706,  0.09404466, -0.368715  ,
         0.06877721, -0.19501355, -0.09041081,  0.324339  , -0.33837777,
         0.21389821,  0.11178841,  0.08214627, -0.38563493, -0.22539575,
        -0.2957237 ,  0.16205397,  0.15159589,  0.05251838, -0.08728514,
         0.07350125,  0.13241772, -0.09883152, -0.193782  , -0.41964212,
         0.21561173, -0.44372752, -0.34466806, -0.18391815, -0.3449145 ,
        -0.09615991, -0.16029893, -0.42752847,  0.02910222,  0.25768444,
         0.13514543, -0.05203221,  0.1031269 , -0.20545632,  0.06002689,
        -0.17601763,  0.01748725,  0.20565306, -0.00641373, -0.05154385,
         0.17976184,  0.13412824, -0.18195182,  0.22584501, -0.12621479,
         0.48221502, -0.3527748 ,  0.04064379, -0.4

In [24]:
#dataset spitting
y_en = df_en['label']
X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(x_train, y_en, test_size=0.3, random_state=100)

In [25]:
# data preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

scaler =  MinMaxScaler()
scaler.fit(X_train_en)
X_train_en = scaler.transform(X_train_en)
X_test_en = scaler.transform(X_test_en)

## Naive Bayes Classifier

In [14]:
model = MultinomialNB()

model.fit(X_train_en,y_train_en)
predictions = model.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.34      1.00      0.51       703
           1       0.00      0.00      0.00       667
           2       0.00      0.00      0.00       691

    accuracy                           0.34      2061
   macro avg       0.11      0.33      0.17      2061
weighted avg       0.12      0.34      0.17      2061

Confusion matrix:
[[703   0   0]
 [666   0   1]
 [691   0   0]]


C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Linear SVM

In [26]:
# without pipeline
clf = LinearSVC(C=1.0, random_state=42)
clf.fit(X_train_en, y_train_en)

y_pred = clf.predict(X_test_en)

print('Accuracy %s' % accuracy_score(y_test_en, y_pred))
print('F1-score %s' % f1_score(y_test_en, y_pred, average=None))
print(classification_report(y_test_en, y_pred))

Accuracy 0.3939835031538088
F1-score [0.4529703  0.32335329 0.38444278]
              precision    recall  f1-score   support

           0       0.40      0.52      0.45       703
           1       0.38      0.28      0.32       667
           2       0.40      0.37      0.38       691

    accuracy                           0.39      2061
   macro avg       0.39      0.39      0.39      2061
weighted avg       0.39      0.39      0.39      2061



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [29]:
#with pipeline and grid serach
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__C': [0.1, 0.5, 1, 10]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train_en,y_train_en)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [30]:
SVM_opt_search.best_params_

{'learner__C': 1, 'sel__k': 50}

In [31]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.39      0.52      0.45       703
           1       0.40      0.27      0.32       667
           2       0.39      0.38      0.39       691

    accuracy                           0.39      2061
   macro avg       0.39      0.39      0.38      2061
weighted avg       0.39      0.39      0.39      2061

Confusion matrix:
[[368 132 203]
 [286 179 202]
 [286 142 263]]


In [33]:
from sklearn.svm import SVC

In [35]:
clf = clf = SVC(gamma='auto', C=0.1, kernel='rbf', random_state=42)
clf.fit(X_train_en, y_train_en)

y_pred = clf.predict(X_test_en)

print('Accuracy %s' % accuracy_score(y_test_en, y_pred))
print('F1-score %s' % f1_score(y_test_en, y_pred, average=None))
print(classification_report(y_test_en, y_pred))

Accuracy 0.3410965550703542
F1-score [0.50868307 0.         0.        ]
              precision    recall  f1-score   support

           0       0.34      1.00      0.51       703
           1       0.00      0.00      0.00       667
           2       0.00      0.00      0.00       691

    accuracy                           0.34      2061
   macro avg       0.11      0.33      0.17      2061
weighted avg       0.12      0.34      0.17      2061



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
#with pipeline and grid serach
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__C': [0.1, 0.5, 1, 10]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', SVC(kernel= 'rbf'))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train_en,y_train_en)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [38]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.35      0.54      0.43       703
           1       0.36      0.23      0.28       667
           2       0.39      0.32      0.35       691

    accuracy                           0.37      2061
   macro avg       0.37      0.36      0.35      2061
weighted avg       0.37      0.37      0.36      2061

Confusion matrix:
[[380 145 178]
 [346 152 169]
 [347 120 224]]


## Random Forest

In [18]:
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__n_estimators': [200, 300, 500, 700]}]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', RandomForestClassifier(bootstrap = True))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=True).fit(X_train_en,y_train_en)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [19]:
opt_search.best_params_

{'learner__n_estimators': 700, 'sel__k': 50}

In [20]:
opt_predictions = opt_search.best_estimator_.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.34      0.47      0.39       703
           1       0.39      0.27      0.32       667
           2       0.34      0.30      0.32       691

    accuracy                           0.35      2061
   macro avg       0.36      0.35      0.34      2061
weighted avg       0.36      0.35      0.35      2061

Confusion matrix:
[[332 151 220]
 [302 183 182]
 [348 136 207]]
